<a href="https://colab.research.google.com/github/LIVECODING-2025/Saruto/blob/main/Analisis_Sentimen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

from tqdm import tqdm
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from transformers import MarianMTModel, MarianTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import nltk
import string
import re

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file_id = '/content/drive/MyDrive/Data Science/Live Coding 2025/komentar_youtube.csv'

In [11]:
df = pd.read_csv(file_id)
print(df.shape)
df.head()

(6000, 4)


,author,komentar,publish,suka
0,@YaniAnggraeni-u1f,Sangat keren! Meraih kemenangan besar di🚦𝐏 𝐋 𝐔...,2025-05-10T05:35:20Z,0
1,@AndrianaIin,🔥𝗣 𝗟 𝗨 𝗧 𝗢 𝟴 𝟴 🔥 memang layak jadi pilihan utama,2025-05-10T05:30:01Z,0
2,@bengal192,NYESEL PILIH PRABOWO,2025-05-10T00:43:29Z,0
3,@khaizuranahda,ekspresinya so iye\npembahasannya walaweee,2025-05-09T20:29:15Z,1
4,@jenifan961,Yang komen kenapa parah banget literasinya?mes...,2025-05-09T18:27:16Z,0


Menghapus kolom yang tidak di gunakan

In [12]:
# Menghapus berdasarkan nama kolom
df = df.drop(columns=['publish', 'suka', 'author'])
df.to_csv("komentar_youtub1.csv", index=False)
print("✅ kolom berhasil di hapus", len(df), "dan disimpan ke 'komentar_youtube1.csv'")

✅ kolom berhasil di hapus 6000 dan disimpan ke 'komentar_youtube1.csv'


In [13]:
df.head()

,komentar
0,Sangat keren! Meraih kemenangan besar di🚦𝐏 𝐋 𝐔...
1,🔥𝗣 𝗟 𝗨 𝗧 𝗢 𝟴 𝟴 🔥 memang layak jadi pilihan utama
2,NYESEL PILIH PRABOWO
3,ekspresinya so iye\npembahasannya walaweee
4,Yang komen kenapa parah banget literasinya?mes...
